<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Creating the Golden Record and Possible Uses
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>


<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>Introduction</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Companies rely on accurate customer valuations to correctly determine how to treat certain classes, what offers to serve them, and how much effort to exert in retaining them. Multi-channel companies often have silos of data from separate sales channels and supplement the data using third-party sources. These multi-channel companies need to combine the data souces to create a holistic view of their customers, particularly their valuation. This demonstration showcases Vantage's capabilities in matching customers from different datasets.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Data analysts match customers across different data sources by calculating the similarity between two names (or other fields) using one of several distance measurements. Steps involved in matching two strings are:</p>

<table style = 'width:100%;table-layout:fixed;'>
    <tr>
        <td style = 'vertical-align:top' width = '50%'>
            <ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
                <li>Select one string to be the base and the other to be the target</li>
                <br>
                <li>Select a string comparison type to use. Examples include Jaro, Jaro-Winkler, Hamming, and cosine</li>
                <br>
                <li>Determine the number of insertions, deletions, replacements, and tranpositions needed to morph the base string into the target string</li>
                <br>
                <li>Calculate the similarity of the two strings by using the formula for the string comparison type chosen and the insertions, deletions, replacements, and transposes</li>
            </ol>
        </td>
        <td><img src = 'images/counting.png' width = '250'></td>
    </tr>
</table>


<b style = 'font-size:16px;font-family:Arial;color:#00233C'>Leverage native Vantage processing for efficient data field matching workflows</b>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>ClearScape Analytics provides a StringSimilarity function to calculate how close two key strings are. Typically, analysts will use different key strings depending on the lists. For example, matching on name works when combining two customer lists, whereas matching on physical addresses works when comparing a customer list with a household list.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Data engineers  combine lists two at a time. For instance, combining four lists A, B, C, and D involves three matching steps. The first step combines A and B to form AB. The second step combines lists AB with C to form ABC. The final step combines ABC with D to form ABCD. Each step requires its own matching table and possibly different key strings.</p>

<img src = 'images/fourlists.png' width = 50% align = "center">


<b style = 'font-size:20px;font-family:Arial;color:#00233C'>Data</b>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The data for this demonstration is based on a brick-and-mortar equipment distributor selling to farmers in four states: New York, Ohio, Pennsylvania, and West Virginia. Over time, this company built a reliable customer list. As customers asked for electronic monitoring service, the company employed an online partner to sell this product. The company also outsourced its call center to a third-party to handle customer service at a cost. Finally, the company worked with a digital marketing group to place cookies on website visitors.</p>


<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The goal is to create a golden record of customers by combining the four lists mentioned above.  Steps in this demonstration are as follows:</p>

<table style = 'width:100%;table-layout:fixed;'>
    <tr>
        <td style = 'vertical-align:top' width = '50%'>
            <ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
                <li>Analyze the raw data from each of the four sources</li>
                <br>
                <li>Append matched online data to the equipment data</li>
                <br>
                <li>Append call center data</li>
                <br>
                <li>Append tagging data</li>
                <br>
                <li>Calculate lifetime value</li>
            </ol>
        </td>
        <td><img src = 'images/process.png' width = '250'></td>
    </tr>
</table>

<hr style="height:2px;border:none;background-color:#00233C;">

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>1. Connect to Vantage </b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>You will be prompted to provide the password. Enter your password, press Enter, then use down arrow to go to next cell.</p>

In [ ]:
%connect local, hidewarnings=true

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Setup for execution of notebook. Begin running steps with Shift + Enter keys.</p>


In [ ]:
Set query_band='DEMO=CustomerIdentityMatching_SQL.ipynb;' update for session;

<hr style="height:2px;border:none;background-color:#00233C;">

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>2. Getting Data for This Demo</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We have provided data for this demo on cloud storage.  You have the option of either running the demo using foreign tables to access the data without using any storage on your environment or downloading the data to local storage which may yield somewhat faster execution, but there could be considerations of available storage.  There are two statements in the following cell, and one is commented out.  You may switch which mode you choose by changing the comment string.</p>


In [ ]:
--call get_data('DEMO_Customer360_cloud'); ---takes about 10 seconds
call get_data('DEMO_Customer360_local'); ---takes about 30 seconds

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Optional step – if you want to see status of databases/tables created and space used.</p>


In [ ]:
call space_report();  -- optional, takes about 10 seconds

<hr style="height:2px;border:none;background-color:#00233C;">

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>3. Analyze Raw Data</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Here, we will inspect each of the four data sets, and determine what fields to use from each table to create a comparison key string.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will inspect:</p>
<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>The equipment distributor's customer table</li>
    <li>The electronic monitor service's e-commerce list relative to the customer table</li>
    <li>The call center list relative to the customer table</li>
    <li>The tagging list relative to the customer table</li>        
</ol>
    
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Replace the following with a valid connection name:</p>

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>3.1- Inspect the Equipment Reseller's Customer Table</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The simple SQL select query below shows that the fields include a customer ID, gender, name, address, and a lifetime value for this channel.</p>

In [ ]:
SELECT TOP 5 * FROM DEMO_Customer360.Equipment

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>3.2- Inspect E-Commerce Table</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>A similar SQL query shows this table has a loyalty number, sex, name, physical address, email address and e-commerce lifetime value. Since both tables' records are at an individual level, we will use first name, last name, and city as a matching key string. Fields to import for now include the loyalty number, and email address.</p>

In [ ]:
SELECT TOP 5 * FROM DEMO_Customer360.Online

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>3.3- Inspect Call Center List</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Note that this table also has records at an individual level, so we can use the same fields to create a matching key. Unique fields to this table include the phone number and call center cost. After matching, we will import the phone number.</p>

In [ ]:
SELECT TOP 5 * FROM DEMO_Customer360.CallCenter

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>3.4 Inspect Cookie List</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Since cookies are collected at the household level and there are no names provided, we will use address as the matching key string. We will append the cookie and campaign to any matching records.</p>

In [ ]:
SELECT TOP 5 * FROM DEMO_Customer360.Cookie

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>3.5 Inspect Customer Across Lists</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will also examine one customer closely to see how his information is matched throughout the lists. This customer is Jessica Rego, and she (or her address) appears in all four tables.  Note that her name is slightly misspelled in the e-commerce (as Jessika), her city is misspelled in call center table (as Nashporte), and her address is misspelled (with no space between 3114 and Park) in the cookie table. </p>

In [ ]:
-- Find Jessica Rego in in-store table
SELECT * FROM DEMO_Customer360.Equipment
WHERE FIRSTNAME = 'Jessica' AND LASTNAME = 'Rego'

In [ ]:
-- Find Jessica Rego in online table
SELECT * FROM DEMO_Customer360.Online
WHERE FNAME LIKE 'Jes%' AND LNAME LIKE 'Reg%'

In [ ]:
-- Find Jessica Rego in call center table
SELECT * FROM DEMO_Customer360.CallCenter
WHERE FIRST_NAME LIKE 'Jes%' AND LAST_NAME LIKE 'Reg%'

In [ ]:
-- Find Jessica Rego's address in tagging table
SELECT * FROM DEMO_Customer360.Cookie
WHERE ADDR_LN_1_TXT LIKE '3114%Park%'

<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>4. Append Online Data to Equipment Table</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>This section will illustrate matching between the equipment and e-commerce tables using the StringSimilarity function built into ClearScape Analytics. The <a href="https://docs.teradata.com/r/Database-Analytic-Functions/July-2021/Advanced-SQL-Engine-Analytic-Functions/StringSimilarity-SQL-Engine">StringSimiliarity documentation</a> contains more information on the function. The StringSimiliarty function supports 10 different ways of comparing two strings. We will use the Jaro similiarity, which accounts for the number of matching characters and transpositions.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Steps for this section include: 
    <img src = 'images/step4.png' width = '350' align='right' padding='padding:50px'></p>

<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
        <li>Exploring which key strings will be matched from a full join</li> 
        <li>Creating a matching table consisting of: 
        <ul><li>The key string from the equipment table</li>
            <li>The key string from the e-commerce table</li>
            <li>The matching score</li>
            <li>Any columns from the e-commerce table we want to append</li>
        </ul></li>
    <li style = 'font-size:16px;font-family:Arial;color:#00233C'>Joining the in-store table with the matching table to form an intermediate golden record</li>
    </ol>
    

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>4.1 - Exploring full join key strings</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will create a key string for each row of the equipment and online tables. The key string will consist of the first name, last name, and city with all spaces removed. Then each key string from the retail table will be compared with each key string in the e-commerce table. The statement below shows the first 20 rows of the comparison. Note that every equipment key is being compared to the first online key.</p>

In [ ]:
SELECT TOP 5
    e.CUST_ID,
    o.LOYALTY_NUM,
    REGEXP_REPLACE(OREPLACE((e.GENDER||e.FIRSTNAME||e.LASTNAME||e.CITY),
                            ' ',''), 
                   '[^a-zA-Z\d:]','',1,0) as EQUIPMENT_KEY,
    REGEXP_REPLACE(OREPLACE((o.SEX||o.FNAME||o.LNAME||o.CITY),
                            ' ',''), 
                   '[^a-zA-Z\d:]','',1,0) as ONLINE_KEY
FROM DEMO_Customer360.Equipment e, DEMO_Customer360.Online o;

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>4.2 - Creating a matching table</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will run the StringSimiliarity function on each row of the table above and only select the rows with a Jaro match of at least a certain threshold. Since the Jaro simliarity score is normalized between 0 and 1, we will choose a threshold of 0.90.</p>

In [ ]:
REPLACE VIEW C360_MatchAB_V AS (
    SELECT * FROM StringSimilarity (
        ON (
            SELECT 
                e.CUST_ID, 
                o.LOYALTY_NUM, 
                CAST(REGEXP_REPLACE(OREPLACE((e.GENDER||e.FIRSTNAME||e.LASTNAME||e.CITY),' ',''), 
                                             '[^a-zA-Z\d:]','',1,0) as VARCHAR(50)) 
                                             as EQUIPMENT_KEY,
                CAST(REGEXP_REPLACE(OREPLACE((o.SEX||o.FNAME||o.LNAME||o.CITY),' ',''), 
                                             '[^a-zA-Z\d:]','',1,0) as VARCHAR(50)) 
                                             as ONLINE_KEY,
                o.email
            FROM DEMO_Customer360.Equipment e, DEMO_Customer360.Online o
        ) PARTITION BY ANY
        USING
        ComparisonColumnPairs(
            'jaro (EQUIPMENT_KEY, ONLINE_KEY) AS jaro'
        )
        CaseSensitive('false')
        Accumulate('CUST_ID', 'LOYALTY_NUM', 'EQUIPMENT_KEY', 'ONLINE_KEY', 'EMAIL')
    ) AS dt 
WHERE jaro > .90)

In [ ]:
SELECT Top 20 * FROM C360_MatchAB_V ORDER BY jaro DESC;

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We see that the first 14 rows consist of exact matches. But starting with the 15th row, the Jaro similarity score is less than 1, indicating there are some discrepancies. For example, taking a look at Jessica Rego, we see that her first name is spelled as Jessika in the online key field:</p>

In [ ]:
SELECT * FROM C360_MatchAB_V WHERE EQUIPMENT_KEY LIKE '%JessicaRego%'

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We find that out of the 100 records in the retail table and 70 in the e-commerce table, 67 were matched.</p>

In [ ]:
SELECT COUNT(*) FROM DEMO_Customer360.Equipment

In [ ]:
SELECT COUNT(*) FROM DEMO_Customer360.Online

In [ ]:
SELECT COUNT(*) FROM C360_MatchAB_V

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>4.3 - Join tables to create intermediate golden record</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Using the CUST_ID and LOYALTY_NUM fields from the above match table, we create an interim golden record from merged retail and e-commerce tables.</p>

In [ ]:
REPLACE VIEW C360_gr1_V AS (
    SELECT e.CUST_ID, mab.LOYALTY_NUM, e.GENDER, 
           e.FIRSTNAME, e.LASTNAME, 
           e.ADDRESS, e.STATE, e.CITY, e.ZIP, 
           mab.EMAIL 
    FROM DEMO_Customer360.Equipment e LEFT JOIN C360_MatchAB_V mab
    ON e.CUST_ID = mab.CUST_ID
)

In [ ]:
SELECT Top 10 * FROM C360_gr1_V;

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We see Jessica's record has been updated with an email address.</p>

In [ ]:
SELECT * FROM C360_gr1_V WHERE FIRSTNAME = 'Jessica' AND LASTNAME = 'Rego'

<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>5. Append Call Center Data</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We repeat the process, this time appending the call center data.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Steps for this section include:
    <img src = 'images/step5.png' width = '300' align='right' padding='padding:50px'></p>

<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>Creating a matching table consisting of:
        <ul><li>The key string from the intermediate golden record table</li>
            <li>The key string from the call center table</li>
            <li>The matching score</li>
            <li>Any columns from the call center table we want to append</li>
        </ul></li>
    <li style = 'font-size:16px;font-family:Arial;color:#00233C'>Joining the intermediate golden record table with the matching table to form a new intermediate golden record</li>
    </ol>

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>5.1 - Creating a matching table</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>As in the previous section, we will create a view consisting of rows with a Jaro match greater than 0.9.</p>

In [ ]:
REPLACE VIEW C360_MatchABC_V AS (
    SELECT * FROM StringSimilarity (
        ON (
            SELECT 
                gr1.CUST_ID, 
                CAST(REGEXP_REPLACE(OREPLACE((gr1.FIRSTNAME||gr1.LASTNAME||gr1.CITY),' ',''), 
                                             '[^a-zA-Z\d:]','',1,0) as VARCHAR(50)) 
                                             as GR1_KEY,
                CAST(REGEXP_REPLACE(OREPLACE((cc.FIRST_NAME||cc.LAST_NAME||cc.CITY),' ',''), 
                                             '[^a-zA-Z\d:]','',1,0) as VARCHAR(50)) 
                                             as CALLCENTER_KEY,
                cc.PHONE                               
            FROM C360_gr1_V gr1, DEMO_Customer360.CallCenter cc
        ) PARTITION BY ANY
        USING
        ComparisonColumnPairs(
            'jaro (GR1_KEY, CALLCENTER_KEY) AS jaro'
        )
        CaseSensitive('false')
        Accumulate('CUST_ID', 'GR1_KEY', 'CALLCENTER_KEY', 'PHONE'/*, 'CALL_CENTER_COST'*/)
    ) AS dt 
WHERE jaro > .90)

In [ ]:
SELECT * FROM C360_MatchABC_V ORDER BY jaro DESC

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We find that out of the 30 in the call center table, 27 were matched.</p>

In [ ]:
SELECT COUNT(*) FROM DEMO_Customer360.CALLCENTER

In [ ]:
SELECT COUNT(*) FROM C360_MatchABC_V

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We see that Jessica Nashport was matched between the two lists even though her last name was misspelled in the Call Center list. The matching score was 0.983.</p>

In [ ]:
SELECT * FROM C360_MatchABC_V WHERE GR1_KEY LIKE '%JessicaRego%'

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>5.2 - Join tables to create intermediate golden record</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Using the CUST_ID, we create another interim golden record including data from the call center table.</p>

In [ ]:
REPLACE VIEW C360_GR2_V AS (
    SELECT gr.CUST_ID, gr.LOYALTY_NUM, gr.GENDER, 
           gr.FIRSTNAME, gr.LASTNAME, 
           gr.ADDRESS, gr.STATE, gr.CITY, gr.ZIP, 
           --gr.EQUIP_LTV, gr.ONLINE_LTV, 
           --COALESCE(mabc.CALL_CENTER_COST, 0) as CALL_CENTER_COST, 
            gr.EMAIL,
            mabc.PHONE
    FROM C360_GR1_V gr LEFT JOIN C360_MatchABC_V mabc
    ON gr.CUST_ID = mabc.CUST_ID
)

In [ ]:
SELECT TOP 10 * FROM C360_GR2_V;

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We see Jessica's record has been updated with an email address (from the last join) and a phone number (from this join).</p>

In [ ]:
SELECT * FROM C360_GR2_V WHERE FIRSTNAME = 'Jessica' AND LASTNAME = 'Rego'

<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>6. Append Cookie Data</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We repeat the process, this time appending the cookie data. Since cookies are collected at the household level, we will use address as the matching string.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Steps for this section include:
<img src = 'images/step6.png' width = '300' align='right' padding='padding:50px'></p>

<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>Creating a matching table
    <li>Joining the intermediate golden record table with the matching table to form a new intermediate golden record</li>
    </ol>

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>6.1 - Creating a matching table</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>As in the previous section, we will create a view consisting of rows with a Jaro match greater than 0.9.</p>

In [ ]:
REPLACE VIEW C360_MatchABCD_V AS (
    SELECT * FROM StringSimilarity (
        ON (
            SELECT 
                gr2.CUST_ID, 
                gr2.EMAIL,
                gr2.PHONE,
                CAST(REGEXP_REPLACE(OREPLACE((gr2.ADDRESS||gr2.CITY),' ',''), 
                                             '[^a-zA-Z\d:]','',1,0) as VARCHAR(50)) 
                                             as GR2_KEY,
                CAST(REGEXP_REPLACE(OREPLACE((t.ADDR_LN_1_TXT||t.CITY_NAME),' ',''), 
                                             '[^a-zA-Z\d:]','',1,0) as VARCHAR(50)) 
                                             as TAGGING_KEY,
                t.COOKIE,
                t.CAMPAIGN
            FROM C360_GR2_V gr2, DEMO_Customer360.Cookie t
        ) PARTITION BY ANY
        USING
        ComparisonColumnPairs(
            'jaro (GR2_KEY, TAGGING_KEY) AS jaro'
        )
        CaseSensitive('false')
        Accumulate('CUST_ID','EMAIL','PHONE', 'GR2_KEY', 'TAGGING_KEY', 'COOKIE','CAMPAIGN')
    ) AS dt 
WHERE jaro > .90)

In [ ]:
SELECT * FROM C360_MatchABCD_V ORDER BY jaro DESC

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We find that out of the 45 in the cookie table, 44 were matched.</p>

In [ ]:
SELECT COUNT(*) FROM DEMO_Customer360.Cookie

In [ ]:
SELECT COUNT(*) FROM C360_MatchABCD_V

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We see that Jessica's physical address was matched between the two lists.</p>

In [ ]:
SELECT * FROM C360_MatchABCD_V WHERE GR2_KEY LIKE '%3114Park%'

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>6.2 - Join tables to create intermediate golden record</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Using the CUST_ID, we create another interim golden record from merged intermediate golden record and cookie tables.</p>

In [ ]:
REPLACE VIEW C360_GR3_V AS (
    SELECT gr2.CUST_ID, gr2.LOYALTY_NUM, gr2.GENDER, 
           gr2.FIRSTNAME, gr2.LASTNAME, 
           gr2.ADDRESS, gr2.STATE, gr2.CITY, gr2.ZIP, gr2.EMAIL,gr2.PHONE,
           mabcd.COOKIE,
           mabcd.CAMPAIGN 
    FROM C360_GR2_V gr2 LEFT JOIN C360_MatchABCD_V mabcd
    ON gr2.CUST_ID = mabcd.CUST_ID
)

In [ ]:
SELECT TOP 5 * FROM C360_GR3_V

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We see that Jessica's record has been updated with a cookie and a campaign.</p>

In [ ]:
SELECT * FROM C360_GR3_V WHERE FIRSTNAME = 'Jessica' AND LASTNAME = 'Rego'

<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>7. Calculate Lifetime Value Across All Channels</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Finally, we calculate the lifetime value by taking into account the channel lifetime value from equipment and online as well as the cost to service a customer from the call center using the formula:<br>
&ensp; LTV = Equipment Purchases + Online Services Sales - Call Center Cost<br>
    Since this is the final calculation, we will write out our results to a table rather than a view.</p>

In [ ]:
CREATE TABLE C360_GoldenRecord AS (
    SELECT gr3.CUST_ID, gr3.LOYALTY_NUM, gr3.GENDER, 
           gr3.FIRSTNAME, gr3.LASTNAME, gr3.EMAIL,gr3.PHONE,
           gr3.ADDRESS, gr3.STATE, gr3.CITY, gr3.ZIP, 
           COALESCE(e.EQUIP_LTV,0) as EQUIPLTV , COALESCE(o.ONLINE_LTV,0) as ONLINELTV, COALESCE(c.CALL_CENTER_COST,0) as CALLCENTER_COST,
            gr3.COOKIE, gr3.CAMPAIGN,
           EQUIPLTV + ONLINELTV - CALLCENTER_COST as LTV
    FROM C360_GR3_V gr3 LEFT JOIN DEMO_Customer360.Equipment e ON gr3.CUST_ID = e.CUST_ID
    LEFT JOIN DEMO_Customer360.Online o ON gr3.EMAIL = o.EMAIL
    LEFT JOIN DEMO_Customer360.CallCenter c ON gr3.PHONE= c.PHONE
    
    )
WITH DATA;

In [ ]:
SELECT TOP 5 * FROM C360_GoldenRecord

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Focusing in on Jessica, we see her lifetime value across all channels.</p>

In [ ]:
SELECT * FROM C360_GoldenRecord WHERE FIRSTNAME = 'Jessica' AND LASTNAME = 'Rego'

<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>8. Conclusion</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Thus by combinig the data from Multiple channels we are able to create a golden record for each customers and calculate the value based on the various channels used. This kind of combined data can be used for various kind of analysis including the one shown in this demo which calculates the life time value for each customers based on the various channels used by the customer and the cost involved.</p>

<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>9. Cleanup</b></p>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Work Tables</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Clean up the work tables to prevent error next time.</p>

In [ ]:
DROP VIEW C360_MatchAB_V;

In [ ]:
DROP VIEW C360_gr1_V; 

In [ ]:
DROP VIEW C360_MatchABC_V;

In [ ]:
DROP VIEW C360_MatchABCD_V; 

In [ ]:
DROP VIEW C360_GR3_V; 

In [ ]:
DROP TABLE C360_GoldenRecord;

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Databases and Tables</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The following code will clean up tables and databases created above.</p>


In [ ]:
call remove_data('DEMO_Customer360'); 

<footer style="padding-bottom:35px; background:#f9f9f9; border-bottom:3px solid #00233C">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2023. All Rights Reserved
        </div>
    </div>
</footer>